PuLPでの汎用問題の作り方
ライブラリのインポート
from pulp import *
数理モデルの作成
最小化問題のとき: m = LpPrblem()
最大化問題のとき: m = LpProblem(sense=LpMaximize)
変数の作成
連続変数: x = LpVariable(変数名, lowBound=0)
0-1変数: x = LpVariable(変数名, cat=LpBinary)
連続変数のリスト: x = [LpVariable(i番目の変数名, lowBound=0) for i in range(n)]
変数名は、必ず異なるようにします
目的関数の設定
m += 式
制約条件の追加
m += 式 == 式
m += 式 <= 式
m += 式 >= 式
式の例
2 * x + 3 * y - 5
和: lpSum(変数のリスト)
内積: lpDot(係数のリスト, 変数のリスト)
ソルバーの実行
m.solve()
変数や式や目的関数の値
value(変数)、value(式)、value(m.objective)

ナップサック問題

In [1]:
#簡単なライブラリ使用
from ortoolpy import knapsack
size = [21, 11, 15, 9, 34, 25, 41, 52]
weight = [22, 12, 16, 10, 35, 26, 42, 53]
capacity = 100
knapsack(size, weight, capacity)

(105.0, [0, 1, 3, 4, 5])

In [4]:
#pulp使用
from pulp import *
size = [21, 11, 15, 9, 34, 25, 41, 52]
weight = [22, 12, 16, 10, 35, 26, 42, 53]
capacity = 100
r = range(len(size))
m = LpProblem(sense = LpMaximize)
x = [LpVariable('x%d'%i, cat = LpBinary) for i in r]
m += lpDot(weight, x)
m += lpDot(size, x) <= capacity
m.solve()
print((value(m.objective), [i for i in r if value(x[i]) > 0.5]))


(105.0, [0, 1, 3, 4, 5])


最短経路問題（ダイクストラ法で解いた方が早いからこの汎用問題としての解き方はオススメしない）

In [2]:
import networkx as nx
g = nx.fast_gnp_random_graph(8, 0.26, 1)
nx.dijkstra_path(g, 0, 2)


[0, 1, 6, 3, 5, 2]

関数の説明
----------
nx.fast_gnp_random_graph
Parameters
----------
n : int
    The number of nodes.
p : float
    Probability for edge creation.
seed : int, optional

In [15]:
from pulp import *
import networkx as nx
g = nx.fast_gnp_random_graph(8,0.26,1).to_directed()
source, sink = 0, 2
r = list(enumerate(g.edges()))  #ノードとノードの連結関係を記述　[1,(1,2)].....
m = LpProblem() # 数理モデル
x = [LpVariable('x%d'%k, lowBound=0, upBound=1) for k, (i, j) in r] # 変数(路に入るかどうか)
m += lpSum(x) # 目的関数
for nd in g.nodes():
    m += lpSum(x[k] for k, (i, j) in r if i == nd) \
      == lpSum(x[k] for k, (i, j) in r if j == nd) + {source:1, sink:-1}.get(nd, 0) # 制約

m.solve()
print([(i,j) for k,(i,j) in r if value(x[k]) > 0.5])
        
    

[(0, 1), (1, 6), (3, 5), (5, 2), (6, 3)]
